Connect To Google Drive.  In this example dataset is a zip file in the specified location. (/content/drive/MyDrive/datasets/dataset.zip)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -o "/content/drive/MyDrive/datasets/dataset.zip" -d "/content/lora_dataset"

# Mounted at /content/drive

Create Virtual Environment

!pip install virtualenv
!virtualenv loraenv

In [ ]:
# every cell that runs code must start with this:
import sys
sys.path.insert(0, '/content/loraenv/lib/python3.10/site-packages')

Install Required Dependencies



In [ ]:
# 1. Clean slate (optional but recommended on fresh runtime)
!pip uninstall -q -y diffusers transformers accelerate peft datasets torch torchvision torchaudio sentence-transformers

# 2. Install PyTorch with correct CUDA (Colab uses CUDA 12.x)
!pip install --no-cache-dir torch torchvision --index-url https://download.pytorch.org/whl/cu121

# 3. Install bleeding-edge diffusers + transformers from GitHub (required for the latest train_lora_sdxl.py)
!pip install -q git+https://github.com/huggingface/diffusers.git
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/accelerate.git
!wget -O train_lora_sdxl.py https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py

# 4. Remaining core dependencies
!pip install -q peft bitsandbytes datasets

Check for any broken dependecies

In [ ]:
!/content/loraenv/bin/pip check

# No broken requirements found.

Generate Caption for each image

In [ ]:
import os
import json

dataset_dir = "/content/lora_dataset"

metadata_path = os.path.join(dataset_dir, "metadata.jsonl")

with open(metadata_path, "w") as f:
    for img_file in os.listdir(dataset_dir):
        if img_file.lower().endswith((".png", ".jpg", ".jpeg")):
            base_name = os.path.splitext(img_file)[0]
            txt_file = base_name + ".txt"
            txt_path = os.path.join(dataset_dir, txt_file)
            
            if os.path.exists(txt_path):
                with open(txt_path, "r") as tf:
                    caption = tf.read().strip()
            else:
                caption = ""  # fallback empty caption
            
            entry = {"file_name": img_file, "text": caption}
            f.write(json.dumps(entry) + "\n")

print(f"Created {metadata_path} with captions from .txt files")

Start Training

In [ ]:
# accelerate command
!accelerate launch /content/train_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="/content/lora_dataset" \
  --output_dir="/content/lora_output" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1200 \
  --mixed_precision="fp16" \
  --gradient_checkpointing \
  --use_8bit_adam \
  --train_text_encoder \
  --seed=42 \
  --validation_prompt="detailed face, masterpiece, best quality" \
  --validation_epochs=50

Save Checkpoint

In [ ]:
import os, glob, shutil

checkpoints = sorted(glob.glob("/content/lora_output/checkpoint-*"), key=os.path.getmtime)
last_ckpt = checkpoints[-1]
print("Last checkpoint:", last_ckpt)

lora_path = os.path.join(last_ckpt, "pytorch_lora_weights.safetensors")
final_lora = "/content/lora_output/trained_lora.safetensors"

shutil.copy(lora_path, final_lora)
print("Saved final LoRA:", final_lora)

Upload trained LoRA to Hugging Face. You'll need to create hugging face token first.

In [ ]:
from huggingface_hub import notebook_login, HfApi, upload_file
notebook_login()

In [ ]:
api = HfApi()
repo_id = "your-username/my-first-lora"   # change to your namespace and repo name
api.create_repo(repo_id, private=False, exist_ok=True)

upload_file(
    path_or_fileobj=final_lora,
    path_in_repo="trained_lora.safetensors",
    repo_id=repo_id
)

print("Uploaded Successfully:", repo_id)

Done!